# ArangoDB NetworkX Adapter Getting Started Guide  

![arangodb](assets/logos/ArangoDB_logo.png) 

In this Notebook we learn how to export Graphs from [ArangoDB](https://www.arangodb.com/), a multi-model Graph Database into [NetworkX](https://networkx.github.io/), the swiss army knife for graph analysis ion python.

# Setup

In [ ]:
!pip install adbnx-adapter
!pip install python-arango
!pip install PyYAML==5.1.1

In [ ]:
#from adbnx_adapter import ArangoDB_Networkx_Adapter

import oasis

# Create a Temporary ArangoDB Instance

In [ ]:
# Request temporary instance from the managed ArangoDB Cloud Oasis.
con = oasis.getTempCredentials()

print()
print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])

*Note: You can also use the above link and login data to explore the ArangoDB UI.*

# The Fraud Detection Dataset

TODO: arangorestore for the import of the imdb Fraud Detection data set

# Connect ArangoDB and NetworkX 

In [ ]:
# Create a ArangoDB_Networkx_Adapter against the temp ArangoDB instance
ma = ArangoDB_Networkx_Adapter(conn = con)
    

In [ ]:
fraud_detection_attributes = { 'vertexCollections': {'Collection1': {'attribute1', 'attribute2'}}, 'edgeCollections' : {'Collection2' : {'att1','att2'}}

g1 = ma.create_networkx_graph(graph_name = 'FraudDetection', graph_attributes = fraud_detection_attributes)

In [ ]:
g.nodes()

In [ ]:
g.edges()